In [1]:
import compute_charge_dist as fz
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

import PeHeat_Functions as peh
import parametric_fz as fzpar

import MCMCFit as mcmc

%matplotlib inline

microntocm    = 1.0e-4
cmtomicron    = 1.0e4
AAtocm        = 1.0e-8
cmtoAA        = 1.0e8
microntoAA    = 1.0e4
AAtomicron    = 1.0e-4
ergtoeV       = 6.242e11
eVtoerg       = 1.602e-12

hplanck       = 4.135667662e-15 # eV s
clight        = 2.99792458e10   # cm s-1

Loading the library to compute the charge distribution of dust grains.


In [2]:
def Cool_Grain(grain_size, grain_type, ZZ, fdist, ntot, xe, temp):
    """
    Compute the cooling per grain.
    """
    import numpy as np
    import math
    import compute_charge_dist as fz

    tau = grain_size * fz.AAtocm * fz.kb * temp / fz.echarge**2
    
    # Loop over species!
    Cool_spec = 0
    for partner in ["electron", "hydrogen", "carbon"]:
    
        Cool_Zall = 0
        # Loop over charge
        for zi in range(len(ZZ)):

            if partner == "electron":
                nu          = -1*ZZ[zi]
                stick_coef  = fz.get_stickCoef(ZZ[zi], grain_size, grain_type)
                charge_frac = xe
                mass        = fz.me
                
            elif partner == "hydrogen":
                nu         = ZZ[zi]
                stick_coef = 1.0
                charge_frac = xe
                mass        = fz.mH
                
            elif partner == "carbon":
                nu         = ZZ[zi]
                stick_coef = 1.0
                # Maximum fraction of ionized Carbon is 1e-4*ntot.
                charge_frac = min(1.0e-4, xe)
                mass        = fz.mC

            Cooltilde = 1.0

            if nu < 0:
                Cooltilde = fz.Jtilde_neg(tau, nu)
            elif nu == 0:
                Cooltilde = fz.Jtilde_0(tau, nu)
            else:
                Cooltilde = fz.Jtilde_pos(tau, nu)

            Cool_Zhere = fdist[zi]*Cooltilde*fz.kb*temp
            Cool_Zall += Cool_Zhere
            
        #print("Collisional Partner = ", partner)
        Cool_spec += ntot*charge_frac*stick_coef*np.sqrt(8.0*fz.kb*temp/(np.pi*mass))*Cool_Zall
        
    Cool_tot = math.pi * (grain_size*AAtocm)**2 * Cool_spec
    
    return Cool_tot

In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

from scipy.optimize import curve_fit

import time

%matplotlib inline


grain_type = "silicate"
pcent      = 1

########################################################################################

grain_size = 3

save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
#save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju/Old_BeforeSept2018"

#filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 
filename = "fz_00%.2iAA_silicate_CR_True_1_pcent.pkl"%(grain_size)

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist3 = pickle.load(pkl_file)
pkl_file.close

# Load Qabs.
Qabs3 = fz.get_QabsTable(grain_type, grain_size, dirtables="/Users/juan/codes/dustanalysis/Charge/Tables")

########################################################################################

grain_size = 5

save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
#save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju/Old_BeforeSept2018"

#filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 
filename = "fz_00%.2iAA_silicate_CR_True_1_pcent.pkl"%(grain_size)

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist5 = pickle.load(pkl_file)
pkl_file.close

# Load Qabs.
Qabs5 = fz.get_QabsTable(grain_type, grain_size, dirtables="/Users/juan/codes/dustanalysis/Charge/Tables")

########################################################################################

grain_size =10

#save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"

#filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 
filename = "fz_00%.2iAA_silicate_CR_True_1_pcent.pkl"%(grain_size)

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist10 = pickle.load(pkl_file)
pkl_file.close

# Load Qabs.
Qabs10 = fz.get_QabsTable(grain_type, grain_size, dirtables="/Users/juan/codes/dustanalysis/Charge/Tables")

########################################################################################

grain_size =50

#save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"

#filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 
filename = "fz_00%.2iAA_silicate_CR_True_1_pcent.pkl"%(grain_size)

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist50 = pickle.load(pkl_file)
pkl_file.close

# Load Qabs.
Qabs50 = fz.get_QabsTable(grain_type, grain_size, dirtables="/Users/juan/codes/dustanalysis/Charge/Tables")


########################################################################################

grain_size =100

#save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"

#filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 
filename = "fz_0%.3iAA_silicate_CR_True_1_pcent.pkl"%(grain_size)

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist100 = pickle.load(pkl_file)
pkl_file.close

# Load Qabs.
Qabs100 = fz.get_QabsTable(grain_type, grain_size, dirtables="/Users/juan/codes/dustanalysis/Charge/Tables")

IOError: [Errno 2] No such file or directory: '/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju/fz_0003AA_silicate_CR_True_1_pcent.pkl'

In [ ]:
########################################################################################

grain_size =100

#save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"
save_dir = "/Users/juan/Dropbox/codes/run/ChargeStatisticsAnalysis/Daikaiju"

#filename = "fz_%.4iAA_%s_CR_True_%i_pcent.pkl"%(grain_size, grain_type, pcent) 
filename = "fz_0%.2iAA_silicate_CR_True_1_pcent.pkl"%(grain_size)

pkl_file    = open("%s/%s"%(save_dir, filename), "rb")
cdist100 = pickle.load(pkl_file)
pkl_file.close

# Load Qabs.
Qabs100 = fz.get_QabsTable(grain_type, grain_size, dirtables="/Users/juan/codes/dustanalysis/Charge/Tables")

---

In [ ]:
print(cdist100["grain_size"])

In [ ]:
Cooling3AA = np.zeros_like(cdist3["nH"], dtype=np.float)

cum = 0
for index in range(len(cdist3["nH"])):

    zmin = cdist3["zminmax"][index*2]
    zmax = cdist3["zminmax"][index*2+1]
    znum = int(abs(zmin) + zmax + 1)

    Gtot = cdist3["G"][index] + cdist3["G_CR"][index]

    ntot = cdist3["nH"][index]
    xe   = cdist3["new_xe"][index]
    temp = cdist3["temp"][index]

    ZZfz = np.arange(zmin, zmax+1, 1)

    ffz = cdist3["fdist"][cum:cum+znum]
    cum +=znum
    
    Cooling3AA[index] = Cool_Grain(3.5, grain_type, ZZfz, ffz, ntot, xe, temp)

In [ ]:
Cooling5AA = np.zeros_like(cdist5["nH"], dtype=np.float)

cum = 0
for index in range(len(cdist5["nH"])):

    zmin = cdist5["zminmax"][index*2]
    zmax = cdist5["zminmax"][index*2+1]
    znum = int(abs(zmin) + zmax + 1)

    Gtot = cdist5["G"][index] + cdist5["G_CR"][index]

    ntot = cdist5["nH"][index]
    xe   = cdist5["new_xe"][index]
    temp = cdist5["temp"][index]

    ZZfz = np.arange(zmin, zmax+1, 1)

    ffz = cdist5["fdist"][cum:cum+znum]
    cum +=znum
    
    Cooling5AA[index] = Cool_Grain(5.0, grain_type, ZZfz, ffz, ntot, xe, temp)

In [ ]:
Cooling10AA = np.zeros_like(cdist10["nH"], dtype=np.float)

cum = 0
for index in range(len(cdist10["nH"])):

    zmin = cdist10["zminmax"][index*2]
    zmax = cdist10["zminmax"][index*2+1]
    znum = int(abs(zmin) + zmax + 1)

    Gtot = cdist10["G"][index] + cdist10["G_CR"][index]

    ntot = cdist10["nH"][index]
    xe   = cdist10["new_xe"][index]
    temp = cdist10["temp"][index]

    ZZfz = np.arange(zmin, zmax+1, 1)

    ffz = cdist10["fdist"][cum:cum+znum]
    cum +=znum
    
    Cooling10AA[index] = Cool_Grain(10.0, grain_type, ZZfz, ffz, ntot, xe, temp)

In [ ]:
Cooling50AA = np.zeros_like(cdist50["nH"], dtype=np.float)

cum = 0
for index in range(len(cdist50["nH"])):

    zmin = cdist50["zminmax"][index*2]
    zmax = cdist50["zminmax"][index*2+1]
    znum = int(abs(zmin) + zmax + 1)

    Gtot = cdist50["G"][index] + cdist50["G_CR"][index]

    ntot = cdist50["nH"][index]
    xe   = cdist50["new_xe"][index]
    temp = cdist50["temp"][index]

    ZZfz = np.arange(zmin, zmax+1, 1)

    ffz = cdist50["fdist"][index]
    cum +=znum
    
    #print("Running cell", index)
    
    #print("Input parameters:", grain_size, grain_type, ZZfz, ffz, ntot, xe, temp, Gtot)
    Cooling50AA[index] = Cool_Grain(50.0, grain_type, ZZfz, ffz, ntot, xe, temp)

In [ ]:
Cooling100AA = np.zeros_like(cdist100["nH"], dtype=np.float)

cum = 0
for index in range(len(cdist100["nH"])):
#for index in range(10):

    zmin = cdist100["zminmax"][index*2]
    zmax = cdist100["zminmax"][index*2+1]
    znum = int(abs(zmin) + zmax + 1)

    Gtot = cdist100["G"][index] + cdist100["G_CR"][index]

    ntot = cdist100["nH"][index]
    xe   = cdist100["new_xe"][index]
    temp = cdist100["temp"][index]

    ZZfz = np.arange(zmin, zmax+1, 1)

    ffz = cdist100["fdist"][index]
    cum +=znum
    
    #print("Running cell", index)
    #print(len(ZZfz), len(ffz))
    #print("Input parameters:", grain_size, grain_type, ZZfz, ffz, ntot, xe, temp, Gtot)
    Cooling100AA[index] = Cool_Grain(100.0, grain_type, ZZfz, ffz, ntot, xe, temp)

 Why is the cooling for a 50A grain not working?????

Find the combination of parameters that reduces the scatter!!!

In [ ]:
#Something like this:

# Take a combination of parameters, bin it with ~ 50 bins, measure the mean of the Cooling in each bin, compute the scatter.
# Vary one of the parameters, do the same and compare the scatters, which one is tighter?

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(131)

GTn3 = (cdist3["G"]+cdist3["G_CR"])*np.sqrt(cdist3["temp"])/cdist3["new_ne"]
BTfit3 = cdist3["new_ne"]*cdist3["temp"]**(0.5)*(GTn3)**(0.2)

ax.hist2d(np.log10(BTfit3),np.log10(Cooling3AA), bins=50, norm=LogNorm(), cmap="inferno")

ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("3.5 $\AA$", fontsize=15)

ax = fig.add_subplot(132)

GTn5 = (cdist5["G"]+cdist5["G_CR"])*np.sqrt(cdist5["temp"])/cdist5["new_ne"]
BTfit5 = cdist5["new_ne"]*cdist5["temp"]**(0.5)*(GTn5)**(0.2)

ax.hist2d(np.log10(BTfit5),np.log10(Cooling5AA), bins=50, norm=LogNorm(), cmap="inferno")

ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("5 $\AA$", fontsize=15)

ax = fig.add_subplot(133)

GTn10 = (cdist10["G"]+cdist10["G_CR"])*np.sqrt(cdist10["temp"])/cdist10["new_ne"]
BTfit10 = cdist10["new_ne"]*cdist10["temp"]**(0.4)*(GTn10)**(0.2)

ax.hist2d(np.log10(BTfit10), np.log10(Cooling10AA), bins=50, norm=LogNorm(), cmap="inferno")

ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("10 $\AA$", fontsize=15)


fig.show()

In [ ]:
fig = plt.figure(figsize=(20,6))
ax = fig.add_subplot(131)

GTn3 = (cdist3["G"]+cdist3["G_CR"])*np.sqrt(cdist3["temp"])/cdist3["new_ne"]
BTfit3 = cdist3["new_ne"]*cdist3["temp"]**(0.5)*(GTn3)**(0.2)

ax.hist2d(np.log10(BTfit3),np.log10(Cooling3AA), bins=50, norm=LogNorm(), cmap="inferno")

ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("3.5 $\AA$", fontsize=15)


ax = fig.add_subplot(132)


GTn3 = (cdist3["G"]+cdist3["G_CR"])*np.sqrt(cdist3["temp"])/cdist3["new_ne"]
BTfit3 = cdist3["new_ne"]*cdist3["temp"]**(0.5)*(GTn3)**(0.2)

ax.hist2d(np.log10(BTfit3),np.log10(Cooling3AA), bins=50, norm=LogNorm(), cmap="inferno")

ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("3.5 $\AA$", fontsize=15)

ax = fig.add_subplot(133)


GTn3 = (cdist3["G"]+cdist3["G_CR"])*np.sqrt(cdist3["temp"])/cdist3["new_ne"]
BTfit3 = cdist3["new_ne"]*cdist3["temp"]**(0.5)*(GTn3)**(0.2)

ax.hist2d(np.log10(BTfit3),np.log10(Cooling3AA), bins=50, norm=LogNorm(), cmap="inferno")

ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("3.5 $\AA$", fontsize=15)

fig.show()

In [ ]:
import MCMCFit as mcmc

In [ ]:
GTn3 = (cdist3["G"]+cdist3["G_CR"])*np.sqrt(cdist3["temp"])/cdist3["new_ne"]
BTfit3 = cdist3["new_ne"]*cdist3["temp"]**(0.5)*(GTn3)**(0.2)

GTn5 = (cdist5["G"]+cdist5["G_CR"])*np.sqrt(cdist5["temp"])/cdist5["new_ne"]
BTfit5 = cdist5["new_ne"]*cdist5["temp"]**(0.5)*(GTn5)**(0.2)

GTn10 = (cdist10["G"]+cdist10["G_CR"])*np.sqrt(cdist10["temp"])/cdist10["new_ne"]
BTfit10 = cdist10["new_ne"]*cdist10["temp"]**(0.5)*(GTn10)**(0.2)

GTn50 = (cdist50["G"]+cdist50["G_CR"])*np.sqrt(cdist50["temp"])/cdist50["new_ne"]
BTfit50 = cdist50["new_ne"]*cdist50["temp"]**(0.5)*(GTn50)**(0.2)

GTn100 = (cdist100["G"]+cdist100["G_CR"])*np.sqrt(cdist100["temp"])/cdist100["new_ne"]
BTfit100 = cdist100["new_ne"]*cdist100["temp"]**(0.5)*(GTn100)**(0.2)

In [ ]:
x3 = np.log10(BTfit3)
y3 = np.log10(Cooling3AA)

x5 = np.log10(BTfit5)
y5 = np.log10(Cooling5AA)

x10 = np.log10(BTfit10)
y10 = np.log10(Cooling10AA)

x50 = np.log10(BTfit50)
y50 = np.log10(Cooling50AA)

x100 = np.log10(BTfit100)
y100 = np.log10(Cooling100AA)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def function(x, m, b):

    y = m*x[:] + b
    
    return y

In [ ]:
popt_3AA, pcov_3AA    = curve_fit(function, x3, y3)
popt_5AA, pcov_5AA    = curve_fit(function, x5, y5)
popt_10AA, pcov_10AA  = curve_fit(function, x10, y10)
popt_50AA, pcov_50AA  = curve_fit(function, x50, y50)
popt_100AA, pcov_100AA  = curve_fit(function, x100, y100)

In [ ]:
print(popt_3AA)
print(popt_5AA)
print(popt_10AA)
print(popt_50AA)
print(popt_100AA)

In [ ]:
mm = [popt_3AA[0], popt_5AA[0], popt_10AA[0],  popt_50AA[0],  popt_100AA[0]]
bb = [popt_3AA[1], popt_5AA[1], popt_10AA[1], popt_50AA[1], popt_100AA[1]]
aa = [3.5, 5., 10., 50., 100.]

In [ ]:
print(np.average(mm))

In [ ]:
fig = plt.figure(figsize=(14,6))
ax = fig.add_subplot(121)

ax.plot(aa, mm)
ax.set_ylim(0, 2)

ax.set_title("m", fontsize=15)

ax = fig.add_subplot(122)

ax.plot(aa, bb)
ax.set_title("b", fontsize=15)

ax.set_xscale("log")

fig.show()

In [ ]:
xsize = np.log10(aa)

In [ ]:
popt_all, pcov_all    = curve_fit(function, xsize, bb)

In [ ]:
print(popt_all)

In [ ]:
newb = 10**(popt_all[1])
print(newb)

y = mx + b

log10(Cool0) = m*(log10 (a)) + b

cool0 = a^(m)*10^(b)

In [ ]:
cool5 = 5**(popt_all[0])*10**(popt_all[1])
print(cool5)

## Cooling:

$$ \Lambda(a) = \Lambda_{0}(a)*(BT94)^{1.17747369697}$$

where:
 
$$ \Lambda_{0}(a) = 4.455\times 10^{-22} \left( \frac{a}{5 AA} \right)^{2.25} $$
 
Where BT94 is:

$$ BT94 = n_{e}*T^{1/2}*(G_{tot}T^{1/2}n_{e}^{-1})^{0.2}$$



In [ ]:
def Cooling_par(asize, Gtot, T, ne):
    """
    Parametric cooling function.
    
    returns cooling rate in:
        erg s-1
    """
    import numpy as np
    
    Lambda0 = 4.455e-22 * (asize/5.0)**(2.24973538)
    psi     = ne * np.sqrt(T) * (Gtot*np.sqrt(T)/ne)**(0.2)
    
    coolhere = Lambda0 * psi**(1.17747369697)
    
    return coolhere

In [ ]:
def Cooling_par_BTin(asize, BT94):
    """
    Parametric cooling function.
    
    returns cooling rate in:
        erg s-1
    """
    import numpy as np
    
    Lambda0 = 4.4552e-22 * (asize/5.0)**(2.24973538)
    
    #BT94 = ne * np.sqrt(T) * (Gtot*np.sqrt(T)/ne)**(0.2)
    
    coolhere = Lambda0 * BT94**(1.17747369697)
    
    return coolhere

In [ ]:
Gtot3 = cdist3["G"] + cdist3["G_CR"]
temp3 = cdist3["temp"]
ne3   = cdist3["new_ne"]

GTn3 = (cdist3["G"]+cdist3["G_CR"])*np.sqrt(cdist3["temp"])/cdist3["new_ne"]
BTfit3 = cdist3["new_ne"]*cdist3["temp"]**(0.5)*(GTn3)**(0.2)


parCool3 = Cooling_par(3.5, Gtot3, temp3, ne3)
parCoolBT = Cooling_par_BTin(3.5,  BTfit3)

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)


ax.hist2d(np.log10(BTfit3),np.log10(Cooling3AA), bins=50, norm=LogNorm(), cmap="inferno", normed=True, weights=GTn3)
ax.plot(np.log10(BTfit3), np.log10(parCoolBT), "-r", linewidth=2)

#ax.hist2d((BTfit3),np.log10(Cooling3AA), bins=50, norm=LogNorm(), cmap="inferno", normed=True, weights=GTn3)
#ax.scatter(BTfit3, np.log10(parCoolBT))


ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(5 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("3.5 $\AA$", fontsize=15)

fig.show()

In [ ]:
Gtot10 = cdist10["G"] + cdist10["G_CR"]
temp10 = cdist10["temp"]
ne10   = cdist10["new_ne"]

GTn10 = (Gtot10)*np.sqrt(temp10)/ne10
BTfit10 = ne10*temp10**(0.5)*(GTn10)**(0.2)


parCool10 = Cooling_par(10,  Gtot10, temp10, ne10)
parCoolBT10 = Cooling_par_BTin(10,  BTfit10)

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)


ax.hist2d(np.log10(BTfit10),np.log10(Cooling10AA), bins=50, norm=LogNorm(), cmap="inferno", normed=True)
ax.plot(np.log10(BTfit10), np.log10(parCoolBT10), "-r", linewidth=2)

#ax.hist2d((BTfit10),np.log10(Cooling10AA), bins=50, norm=LogNorm(), cmap="inferno", normed=True, weights=GTn10)
#ax.scatter(BTfit10, np.log10(parCoolBT10))


ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(10 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("10 $\AA$", fontsize=15)

fig.show()

In [ ]:
Gtot100 = cdist100["G"] + cdist100["G_CR"]
temp100 = cdist100["temp"]
ne100   = cdist100["new_ne"]

GTn100 = (Gtot100)*np.sqrt(temp100)/ne100
BTfit100 = ne100*temp100**(0.5)*(GTn100)**(0.5)


#parCool100 = Cooling_par(100,  Gtot100, temp100, ne100)
#parCoolBT100 = Cooling_par_BTin(100,  BTfit100)

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)


ax.hist2d(np.log10(BTfit100),np.log10(Cooling100AA), bins=50, norm=LogNorm(), cmap="inferno", normed=True)
#ax.plot(np.log10(BTfit100), np.log10(parCoolBT100), "-r", linewidth=2)

#ax.hist2d((BTfit10),np.log10(Cooling10AA), bins=50, norm=LogNorm(), cmap="inferno", normed=True, weights=GTn10)
#ax.scatter(BTfit10, np.log10(parCoolBT10))


ax.set_xlabel("some combination of GTn$_{e}$", fontsize=15)
ax.set_ylabel(" $\Lambda^{'}_{pe}(100 \\AA)$ [ergs]", fontsize=15)

ax.tick_params(axis='both', which='major', length=10, width=2,  labelsize=15, direction="in")
ax.tick_params(axis='both', which='minor', length=5, width=1.5, labelsize=15, direction="in")

ax.set_title("100 $\AA$", fontsize=15)

fig.show()

---

I need to check if this is still valid for carbonaceous grains. I think it should, they use the same equations and there was no distinction between grains in Draine & Sutin 1987.

In [ ]:
Gtot = Gtot10
ne = ne10
T = temp10

In [ ]:
bestfit = True

# Initial guess
alpha = 0.2
beta = 0.8
gamma = 0.7

GTn_comb = Gtot**(alpha)*ne**(beta)*T**(gamma)

logarr = np.logspace(np.log10(np.min(GTn_comb)), np.log10(np.max(GTn_comb)), num=10)

mean = np.zeros_like(logarr, dtype=np.float)
std  = np.zeros_like(logarr, dtype=np.float)

for i in range(len(logarr)-1):
#for i in range(2):
    mask = np.where((GTn_comb >= logarr[i]) & (GTn_comb < logarr[i+1]))
    GTnhere = GTn_comb[mask]
    #print(logarr[i], logarr[i+1])
    #print(len(GTnhere), np.min(GTnhere), np.max(GTnhere))
    mean[i] = np.average(GTnhere)
    std[i] = np.std(GTnhere)
    
mean[-1] = mean[-2]
std[-1] = std[-2]

if bestfit:
    alphabest = alpha
    betabest=beta
    gammabest=gamma
    bestfit = False

In [ ]:
plt.plot(np.log10(logarr), np.log10(mean))
plt.plot(np.log10(logarr), np.log10(std))

In [ ]:
bestfit = True

# Initial guess
alpha = 1.0974368543429029 # 0.2
beta = 0.8
gamma = 0.7

counter = 0

while counter < 500:

    GTn_comb = Gtot**(alpha)*ne**(beta)*T**(gamma)

    logarr = np.logspace(np.log10(np.min(GTn_comb)), np.log10(np.max(GTn_comb)), num=10)

    mean = np.zeros_like(logarr, dtype=np.float)
    std  = np.zeros_like(logarr, dtype=np.float)

    for i in range(len(logarr)-1):
        mask = np.where((GTn_comb >= logarr[i]) & (GTn_comb < logarr[i+1]))
        GTnhere = GTn_comb[mask]
        mean[i] = np.average(GTnhere)
        std[i] = np.std(GTnhere)

    mean[-1] = mean[-2]
    std[-1] = std[-2]

    if bestfit == False:
        if np.average(std) < np.average(beststd):
            bestfit = True
            print("Found a better combination of parameters that minimizes the scatter:")
            print("alpha=", alpha, "beta=", beta, "gamma=", gamma)
        
    if bestfit:
        alphabest = alpha
        betabest=beta
        gammabest=gamma
        
        bestmean= mean
        beststd = std
        
        bestfit = False
        
    # SHould test both positie and negative!!
    #alpha = alpha - 0.1*np.random.rand(1)[0]
    #beta = beta + 0.1*np.random.rand(1)[0]
    gamma = gamma - 0.1*np.random.rand(1)[0]
    
    counter+=1